<a href="https://colab.research.google.com/github/aasthagit2025/DV-Macro/blob/main/DV_Macro_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install necessary libraries
!pip install pyreadstat pandas openpyxl

import pandas as pd
import pyreadstat
from google.colab import files

# ---- Upload Files ----
print("👉 Upload your survey file (CSV or SPSS .sav)")
uploaded = files.upload()
data_file = list(uploaded.keys())[0]

print("👉 Upload your validation rules file (Excel .xlsx)")
uploaded = files.upload()
rules_file = list(uploaded.keys())[0]

# ---- Load Data ----
if data_file.endswith(".sav"):
    df, meta = pyreadstat.read_sav(data_file)
    print("✅ Loaded SPSS file")
elif data_file.endswith(".csv"):
    df = pd.read_csv(data_file)
    print("✅ Loaded CSV file")
else:
    raise ValueError("Unsupported file type. Upload .csv or .sav")

print(f"Data shape: {df.shape[0]} rows × {df.shape[1]} columns")

# ---- Load Validation Rules ----
rules = pd.read_excel(rules_file)
print("✅ Validation rules loaded")

# ---- Validation Functions ----
def validate_row(row, rules):
    errors = []
    for _, rule in rules.iterrows():
        variables = rule['variable'].split(',')
        rtype = rule['rule_type']
        param = str(rule['parameter'])
        message = rule['message']

        if rtype == "range":
            min_val, max_val = map(float, param.split('-'))
            for v in variables:
                if pd.notna(row[v]) and not (min_val <= row[v] <= max_val):
                    errors.append(f"{v}: {message} (found {row[v]})")

        elif rtype == "range_exclude":
            range_part, exclude_part = param.split(';')
            min_val, max_val = map(float, range_part.split('-'))
            exclude_vals = list(map(float, exclude_part.replace('exclude=','').split(',')))
            for v in variables:
                if pd.notna(row[v]) and row[v] not in exclude_vals and not (min_val <= row[v] <= max_val):
                    errors.append(f"{v}: {message} (found {row[v]})")

        elif rtype == "categorical":
            allowed = list(map(float, param.split(',')))
            for v in variables:
                if pd.notna(row[v]) and row[v] not in allowed:
                    errors.append(f"{v}: {message} (found {row[v]})")

        elif rtype == "text_length":
            max_len = int(param.replace('max=',''))
            for v in variables:
                if isinstance(row[v], str) and len(row[v]) > max_len:
                    errors.append(f"{v}: {message} (length {len(row[v])})")

        elif rtype == "skip_check":
            cond_var, cond_val = param.split('=')
            cond_val = float(cond_val)
            if row[cond_var] == cond_val:
                for v in variables:
                    if pd.notna(row[v]):
                        errors.append(f"{v}: {message} (found {row[v]})")

        elif rtype == "consistency_check":
            cond_var, cond_val = param.split('=')
            cond_val = float(cond_val)
            for v in variables:
                if row[cond_var] == cond_val and pd.isna(row[v]):
                    errors.append(f"{v}: {message}")

        elif rtype == "straightliner_check":
            min_items = int(param.replace('min_items=',''))
            responses = [row[v] for v in variables if pd.notna(row[v])]
            if len(responses) >= min_items and len(set(responses)) == 1:
                errors.append(f"{','.join(variables)}: {message} (all {responses[0]})")

    return errors

# ---- Run Validation ----
all_errors = {}
for idx, row in df.iterrows():
    row_errors = validate_row(row, rules)
    if row_errors:
        all_errors[row.get("RespondentID", idx)] = row_errors

# ---- Duplicate Check ----
duplicate_vars = rules[rules['rule_type']=='duplicate_check']['variable'].tolist()
# Correct the column name from 'RespID' to 'RespondentID' if it exists in duplicate_vars
if 'RespID' in duplicate_vars:
    duplicate_vars[duplicate_vars.index('RespID')] = 'RespondentID'

for dup_var in duplicate_vars:
    dupes = df[df.duplicated(subset=[dup_var], keep=False)]
    for val in dupes[dup_var].unique():
        all_errors.setdefault(val, []).append(f"{dup_var}: Duplicate value found")


# ---- Output Results ----
results_df = pd.DataFrame([
    {"RespondentID": rid, "Error": e}
    for rid, errs in all_errors.items()
    for e in errs
])

output_file = "validation_results.xlsx"
results_df.to_excel(output_file, index=False)
print("✅ Validation complete. Download your results below:")
files.download(output_file)

👉 Upload your survey file (CSV or SPSS .sav)


Saving sample_data.csv to sample_data (1).csv
👉 Upload your validation rules file (Excel .xlsx)


Saving validation_rules.xlsx to validation_rules (1).xlsx
✅ Loaded CSV file
Data shape: 5 rows × 13 columns
✅ Validation rules loaded
✅ Validation complete. Download your results below:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>